# Chem Informatics Tutorial on nfp and GNNs

02nd Nov 2022

Shree Sowndarya S V

This collab can be run remoting using google's computing resources as well as on local machines. Most of the comments would remain the same aside for enviroment setup i.e., the installation of software blocks.

# Necessary packages to install for this code to run

*   Step 1: PIP installation of necessary packages


In [23]:
%%time
#install nfp
# git clone https://github.com/shreesowndarya/nfp.git -b shree_changes
# python setup.py install

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.72 µs


# Necessary datasets we would be working with 

*   Step 1: Loading the dataset from github
*   Step 2: Split the data for train and test


In [24]:
# Step 1: Loading the dataset from github
import pandas as pd
url = 'https://raw.githubusercontent.com/shreesowndarya/HRC/main/reduced_radical_db_redox_potentials.csv'
data = pd.read_csv(url)
data.head()

,smiles,oxidation,reduction
0,C[C](CO)CCC#N,0.027999,-1.660134
1,CC1=CC[C](O)CC1,-0.770340,-1.920379
2,C=CCCC[CH]CCN,0.231808,-1.867590
3,C[C](C)NCCCN,-1.777168,-2.273575
4,[CH2]O/C=C/C(=O)OC,0.374692,-1.128816


In [25]:
# Step 2: Split the data for train and test
import numpy as np
valid, test, train = np.split(data.smiles.sample(frac=1., random_state=1), [100, 200])
print('Training Set:', len(train))
print('Validation Set:' , len(valid))
print('Testing Set:' , len(test))

Training Set: 2097
Validation Set: 100
Testing Set: 100


# nfp example

*   Step 1: Import nfp for processing smiles to graphs
*   Step 2: Define the kind of features we are using for atoms and bonds in the molecule
*   Step 3: Example showing the features utilized for atoms and bonds
*   Step 4: create a nfp graph processor using the feature we need
*   Step 5: Example of graphical inputs for SMILES - 'CCO'
*   Step 6: Graphical inputs for a dataset of SMILES- What do the numbers mean?

In [26]:
# Step 1:
from nfp.preprocessing.features import get_ring_size

In [27]:
# Step 2: Definition the kind of features we are using for atoms and bonds in the molecule
def atom_featurizer(atom):
    """ Return a string representing the atom type 
    
    returns [symbol, is_aromatic, ring_size, degree, total_num_Hs]
    """

    return str((
        atom.GetSymbol(),
        atom.GetIsAromatic(),
        get_ring_size(atom, max_size=6),
        atom.GetDegree(),
        atom.GetTotalNumHs(includeNeighbors=True)
    ))

def bond_featurizer(bond, flipped=False):
    """ Get a similar classification of the bond type.
    Flipped indicates which 'direction' the bond edge is pointing.

    returns [atoms_in_bond, bond_type, ring_size]
    """
    
    if not flipped:
        atoms = "{}-{}".format(
            *tuple((bond.GetBeginAtom().GetSymbol(),
                    bond.GetEndAtom().GetSymbol())))
    else:
        atoms = "{}-{}".format(
            *tuple((bond.GetEndAtom().GetSymbol(),
                    bond.GetBeginAtom().GetSymbol())))
    
    btype = str(bond.GetBondType())
    ring = 'R{}'.format(get_ring_size(bond, max_size=6)) if bond.IsInRing() else ''
    
    return " ".join([atoms, btype, ring]).strip()

In [28]:
# Step 3: Example showing the features utilized for atoms and bonds
from rdkit import Chem

smiles = 'CCO'
mol = Chem.MolFromSmiles(smiles)
print('Atom features:', atom_featurizer(mol.GetAtoms()[0]))
print('Bond features:', bond_featurizer(mol.GetBonds()[1]))

Atom features: ('C', False, 0, 1, 3)
Bond features: C-O SINGLE


In [29]:
# Step 4: create a nfp graph processor using the feature we need
from nfp.preprocessing.mol_preprocessor import SmilesPreprocessor
preprocessor = SmilesPreprocessor(atom_features=atom_featurizer, bond_features=bond_featurizer, explicit_hs=False)

In [30]:
# Step 5: Example of graphical inputs for SMILES - 'CCO'
print('Atoms Classes:', preprocessor(smiles, train=True)['atom']) # Atom types, as integer classes
print('Bond Classes:', preprocessor(smiles, train=True)['bond']) # Bond types, as integer classes
print('Connectivity:', preprocessor(smiles, train=True)['connectivity']) # A connectivity array, where row i 
                                                                  #indicates bond i connects atom j to atom k

Atoms Classes: [2 3 4]
Bond Classes: [2 2 3 4]
Connectivity: [[0 1]
 [1 0]
 [1 2]
 [2 1]]


In [31]:
# Step 6: Graphical inputs for a dataset of SMILES - What do the numbers mean?
for smiles in data.smiles: 
    preprocessor(smiles, train=True)

print('Atom data types: \n', pd.Series(preprocessor.atom_tokenizer._data))
print('\nBond data types: \n', pd.Series(preprocessor.bond_tokenizer._data))

Atom data types: 
 unk                           1
('C', False, 0, 1, 3)         2
('C', False, 0, 2, 2)         3
('O', False, 0, 1, 1)         4
('C', False, 0, 3, 0)         5
                             ..
('S', False, 'max', 3, 0)    88
('S', False, 3, 2, 0)        89
('S', False, 4, 2, 0)        90
('C', True, 3, 3, 0)         91
('N', False, 3, 4, 0)        92
Length: 92, dtype: int64

Bond data types: 
 unk                  1
C-C SINGLE           2
C-O SINGLE           3
O-C SINGLE           4
C-N TRIPLE           5
                  ... 
O-S SINGLE R5      116
S-O SINGLE R5      117
N-N DOUBLE R3      118
C-C TRIPLE Rmax    119
C-C AROMATIC R3    120
Length: 120, dtype: int64


# Building Graphs with xTB features

*   Step 1: Run xtb calculation to obtain properties as a json file (example shown)
*   Step 2: Use xTB preprocessor to build graphs with xtb features added to atoms and nodes
*   Step 3: Compile and construct the GNNs

In [32]:
#readint the data with the links to xtbjson
url = 'https://raw.githubusercontent.com/shreesowndarya/ChemInformatics/main/Fall_2022/ysi_xtb.csv'
data2 = pd.read_csv(url)
data2.head()

,Species,CAS,Ref,Type,YSI,YSI_err,SMILES,xtbjson
0,"1-ethynyl-2,5-dimethylbenzene",74331-70-7,1.0,aromatic,512.7,19.6,C#Cc1cc(C)ccc1C,519175_3342b2_bd_coord.json
1,2-methylindene,2177-47-1,1.0,aromatic,500.1,19.1,CC1=Cc2ccccc2C1,437628_fd0a51_bd_coord.json
2,azulene,275-51-4,2.0,aromatic,492.3,19.0,c1ccc2cccc-2cc1,435907_1c36e8_bd_coord.json
3,1-ethynyl-2-methylbenzene,766-47-2,1.0,aromatic,485.0,18.5,C#Cc1ccccc1C,22307_0_bd_coord.json
4,"1-ethenyl-2,5-dimethylbenzene",2039-89-6,1.0,aromatic,469.3,17.8,C=Cc1cc(C)ccc1C,509114_ad5711_bd_coord.json


In [33]:
# Split the data into training, validation, and test sets
valid2, test2, train2 = np.split(data2[['SMILES','xtbjson']].sample(frac=1., random_state=1), [50, 100])
len(train2), len(valid2), len(test2)

(134, 50, 50)

In [34]:
#loading in xTB preprocessor
from nfp.preprocessing.xtb_preprocessor import xTBSmilesPreprocessor
preprocessorxtb = xTBSmilesPreprocessor(atom_features=atom_featurizer, bond_features=bond_featurizer)

In [35]:
# Main input types for a SMILES-based prediction
smiles = 'C=C(C)CC(C)(C)C'
jsonfile = 'json/4921_0_bd_coord.json'

# Atom types, as integer classes
xtb_inputs = preprocessorxtb(smiles, jsonfile=jsonfile, train=True)

In [36]:
xtb_inputs

{'atom_indices': array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23], dtype=int32),
 'atom': array([2, 3, 4, 5, 6, 4, 4, 4, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
        7, 7], dtype=int32),
 'atom_xtb': array([[-2.0606928 , -1.6120849 , -1.3153727 , -0.8517737 , -1.191078  ,
          1.3394293 ,  1.4466555 ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  2.0678065 ,  2.181046  ],
        [ 0.4309656 , -0.02760521, -2.3100824 , -0.44524527, -1.6921847 ,
          2.6923866 ,  1.3149185 ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.7533343 ,  1.9715408 ],
        [-1.6546047 , -1.7723731 ,  1.546599  ,  1.9939245 ,  1.7927859 ,
          0.44647667,  1.4543637 ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  1.2734121 ,  1.1803653 ],
        [-1.1914048 , -1.2503973 ,  1.1801271 ,  1.2099056 ,  1.2461238 ,
          1.2311907 ,  1.41582

In [37]:
# can use xTBSmilesPreprocessor, xTBWBOPreprocessor, xTB3DPreprocessor

# Building Graph Neural Networks (GNNs)

*   Step 1: Convert the graphs into tensorflow syntax for builing GNN - Here we are considering only Oxidation to keep things simple
*   Step 2: Define the neural network model 
*   Step 3: Compile and construct the GNNs

In [38]:
# Step 1: Convert the graphs into tensorflow syntax for builing GNN
import tensorflow as tf

train_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.smiles, train=True), row['oxidation'])
            for i, row in data[data.smiles.isin(train)].iterrows()),
    output_signature=(preprocessor.output_signature, tf.TensorSpec((), dtype=tf.float32)))\
    .cache().shuffle(buffer_size=200)\
    .padded_batch(batch_size=(128))\
    .prefetch(tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.Dataset.from_generator(
    lambda: ((preprocessor(row.smiles, train=False), row['oxidation'])
            for i, row in data[data.smiles.isin(valid)].iterrows()),
    output_signature=(preprocessor.output_signature, tf.TensorSpec((), dtype=tf.float32)))\
    .cache()\
    .padded_batch(batch_size=(64))\
    .prefetch(tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.Dataset.from_generator(
  lambda: (preprocessor(smiles, train=False)
          for smiles in test),
  output_signature=preprocessor.output_signature)\
  .padded_batch(batch_size=len(test))\
  .prefetch(tf.data.experimental.AUTOTUNE)

In [39]:
# Step 2: Define the neural network model
from tensorflow.keras import layers
import nfp

# Input layers
atom = layers.Input(shape=[None], dtype=tf.int64, name='atom')
bond = layers.Input(shape=[None], dtype=tf.int64, name='bond')
connectivity = layers.Input(shape=[None, 2], dtype=tf.int64, name='connectivity')

num_features = 8  # Controls the size of the model

# Convert from a single integer defining the atom state to a vector
# of weights associated with that class
atom_state = layers.Embedding(preprocessor.atom_classes, num_features,
                              name='atom_embedding', mask_zero=True)(atom)

# Ditto with the bond state
bond_state = layers.Embedding(preprocessor.bond_classes, num_features,
                              name='bond_embedding', mask_zero=True)(bond)

# Here we use our first nfp layer. This is an attention layer that looks at
# the atom and bond states and reduces them to a single, graph-level vector. 
# mum_heads * units has to be the same dimension as the atom / bond dimension
global_state = nfp.GlobalUpdate(units=8, num_heads=1)([atom_state, bond_state, connectivity])

for _ in range(3):  # Do the message passing
    new_bond_state = nfp.EdgeUpdate()([atom_state, bond_state, connectivity, global_state])
    bond_state = layers.Add()([bond_state, new_bond_state])
    
    new_atom_state = nfp.NodeUpdate()([atom_state, bond_state, connectivity, global_state])
    atom_state = layers.Add()([atom_state, new_atom_state])
    
    new_global_state = nfp.GlobalUpdate(units=8, num_heads=1)(
        [atom_state, bond_state, connectivity, global_state]) 
    global_state = layers.Add()([global_state, new_global_state])

# Since the final prediction is a single, molecule-level property (oxidation, reduction), we 
# reduce the last global state to a single prediction
ox_prediction = layers.Dense(1)(global_state)

# Construct the tf.keras model
model = tf.keras.Model([atom, bond, connectivity], [ox_prediction])

In [40]:
# # Step 3: Compile and construct the GNNs
model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(1E-3))
model.fit(train_dataset, validation_data=valid_dataset, epochs=100)

Epoch 1/100
17/17 [==============================] - 8s 211ms/step - loss: 0.7192 - val_loss: 0.7199
Epoch 2/100
17/17 [==============================] - 0s 11ms/step - loss: 0.6337 - val_loss: 0.5945
Epoch 3/100
17/17 [==============================] - 0s 10ms/step - loss: 0.5625 - val_loss: 0.5857
Epoch 4/100
17/17 [==============================] - 0s 11ms/step - loss: 0.5183 - val_loss: 0.5286
Epoch 5/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4840 - val_loss: 0.5214
Epoch 6/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4637 - val_loss: 0.5001
Epoch 7/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4448 - val_loss: 0.4702
Epoch 8/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4347 - val_loss: 0.4435
Epoch 9/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4215 - val_loss: 0.4280
Epoch 10/100
17/17 [==============================] - 0s 11ms/step - loss: 0.4099 - val_loss: 0.411

17/17 [==============================] - 0s 12ms/step - loss: 0.1247 - val_loss: 0.2080
Epoch 83/100
17/17 [==============================] - 0s 11ms/step - loss: 0.1152 - val_loss: 0.2075
Epoch 84/100
17/17 [==============================] - 0s 11ms/step - loss: 0.1093 - val_loss: 0.2076
Epoch 85/100
17/17 [==============================] - 0s 11ms/step - loss: 0.1102 - val_loss: 0.2100
Epoch 86/100
17/17 [==============================] - 0s 12ms/step - loss: 0.1163 - val_loss: 0.2134
Epoch 87/100
17/17 [==============================] - 0s 12ms/step - loss: 0.1134 - val_loss: 0.1984
Epoch 88/100
17/17 [==============================] - 0s 12ms/step - loss: 0.1129 - val_loss: 0.2141
Epoch 89/100
17/17 [==============================] - 0s 12ms/step - loss: 0.1297 - val_loss: 0.2156
Epoch 90/100
17/17 [==============================] - 0s 12ms/step - loss: 0.1209 - val_loss: 0.2002
Epoch 91/100
17/17 [==============================] - 0s 11ms/step - loss: 0.1152 - val_loss: 0.2031
Epo

# Testing Graph Neural Networks (GNNs)

*   Step 1: Predict using the built model
*   Step 2: Plot to see test set





In [41]:
# Step 1: Predict using the built model
test_predictions = model.predict(test_dataset)
test_db_values = data.set_index('smiles').reindex(test)['oxidation'].values

In [42]:
# Step 2: Plot to see test set
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import r2_score
import sklearn.metrics as metrics

fig = plt.subplots()

ax1 = sns.scatterplot(test_db_values,test_predictions.flatten(),s=30,marker='o',color='b',alpha=0.5)
ax1.set_xlabel(r'Calculated Oxidation in eV',fontsize=10)
ax1.set_ylabel(r'Predicted Oxidation in eV',fontsize=10)

mae = metrics.mean_absolute_error(test_db_values,test_predictions.flatten())
r2 = metrics.r2_score(test_db_values,test_predictions.flatten())

plt.annotate(f"$R^2$ = {round(r2,1)} \nMAE = {round(mae,1)} ", xy=(1.0, -1.5), fontsize=14)

/opt/anaconda3/envs/my-rdkit-env/lib/python3.7/site-packages/seaborn/_decorators.py:43: FutureWarning: Pass the following variables as keyword args: x, y. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  FutureWarning


Text(1,-1.5,'$R^2$ = 0.9 \nMAE = 0.2 ')